# Get Vacancy by Radius

### 01/01/18

- get vacantOSMNX_1516KenhillAve2000_010118

### 12/31/17

- get vacantOSMNX_1516KenhillAve1800_123117

### 12/30/17

- get vacantOSMNX_1516KenhillAve1200_123017

- get vacantOSMNX_1516KenhillAve1300_123017

### 12/29/17

- get vacantOSMNX_1516KenhillAve1000_122917

- get vacantOSMNX_1516KenhillAve1100_122917

### 12/28/17

- delete Set Edge

- get vacantOSMNX_1516KenhillAve800_122817

- get vacantOSMNX_1516KenhillAve900_122917

In [1]:
# get clean vacant
import csv
import pandas as pd
import time
vacant_Baltimore = pd.read_csv('CleanDataBase/vacantBaltimore_171117_8000.csv',sep='\t') # create header later
vacant_Baltimore = vacant_Baltimore.drop("Unnamed: 0", 1)

# clean street name
vacant_Baltimore.loc[:,"street"] = vacant_Baltimore.loc[:,"street"].str[1:-1]
print "number of Vacancy in db25:", len(vacant_Baltimore)

number of Vacancy in db25: 8602


In [2]:
# get osmnx GEOdataframe
import osmnx as ox
address = '1516 Kenhill Ave, Baltimore, MD'
radius = 2000
gdf = ox.buildings_from_address(address, distance=radius)

print "Address:", address
print "radius", radius
print "number of buildings from osmnx:", len(gdf)

Address: 1516 Kenhill Ave, Baltimore, MD
radius 2000
number of buildings from osmnx: 34200


In [3]:
## check iff street is not null and building type is yes
result = pd.concat([gdf["addr:street"].isnull() == False, gdf["building"] == "yes" ], axis=1)
result["sum"] = result["addr:street"] & result["building"]
street_gdf = gdf[result["sum"]]

print "number of buildings with street name is not null:", len(street_gdf)

number of buildings with street name is not null: 29791


In [4]:
def vacant_generator(i,j):
    
    while i < len(street_gdf.index):
        while j < len(vacant_Baltimore.index):
            if (street_gdf['addr:street'][street_gdf.index[i]] == vacant_Baltimore['street'][vacant_Baltimore.index[j]] and 
                street_gdf['addr:housenumber'][street_gdf.index[i]] == vacant_Baltimore['housenumber'][vacant_Baltimore.index[j]]):
                
                yield street_gdf.index[i],vacant_Baltimore.index[j], street_gdf['addr:street'][street_gdf.index[i]], street_gdf['addr:housenumber'][street_gdf.index[i]]
            yield street_gdf.index[i],vacant_Baltimore.index[j],None,None
            j += 1
        
        if i%1000==0:
            print i
            
        i += 1
        j = 0
        
                #m.gdf.loc[i,'housetype'] = 2
            #elif m.gdf['housetype'][i] == 2:
                #m.gdf.loc[i,'housetype'] = 1

In [5]:
vacantOSMNX = []
vacantdb25 = []
s = time.time()
for i,j,street,housenumber in vacant_generator(0,0):
    if street != None:
        print street, housenumber, i, j
        vacantOSMNX.append(i)
        vacantdb25.append(j)
print "get", len(vacantOSMNX), "houses using", time.time() - s, "s"

0
Pulaski Highway 4006 188579014 1439
Pulaski Highway 4023 188579025 6486
Chatford Avenue 4720 336190755 3990
Roberton Avenue 4317 336191721 1910
Shannon Drive 4121 336192110 7435
Shamrock Avenue 4710 336192558 6536
Roberton Avenue 4312 336192668 5664
Ardley Avenue 4112 336193299 8334
Shamrock Avenue 4645 336194468 2898
Ardley Avenue 4027 336194945 5532
Parkside Drive 4626 336194998 5019
Raymonn Avenue 4025 336195873 5954
Balfern Avenue 4108 336196151 8361
1000
South Kresson Street 3 336197915 4571
South Kresson Street 9 336197935 6590
South Kresson Street 1 336197939 885
North Kresson Street 917 336198572 7663
North Janney Street 43 336198955 3694
East Lombard Street 4310 336198996 3973
East Lombard Street 4318 336199005 4858
South Janney Street 5 336199372 1997
South Janney Street 13 336199376 1990
Shannon Drive 3573 336199545 2681
North Janney Street 114 336199726 5232
North Janney Street 132 336199736 1534
Nicholas Avenue 4387 336199785 468
Dudley Avenue 3669 336200346 7264
Sheldon

Lake Avenue 2826 336236164 5840
Lake Avenue 2840 336236177 428
Lawnview Avenue 3229 336236210 1862
Lawnview Avenue 3203 336236224 2869
North East Avenue 130 336236315 2299
East Monument Street 3127 336236437 1293
East Monument Street 3123 336236439 5799
North Robinson Street 639 336236451 7402
North Robinson Street 603 336236461 2445
North Robinson Street 439 336236537 3461
North Robinson Street 455 336236539 4213
East Fairmount Avenue 3100 336236561 5409
Edison Highway 2914 336236592 2314
Edison Highway 2932 336236599 2882
Cliftmont Avenue 3218 336236719 6984
East Fayette Street 3105 336236741 5264
Cardenas Avenue 3524 336237062 1090
Cardenas Avenue 3528 336237066 424
Pelham Avenue 2755 336237128 1569
North Robinson Street 322 336237402 2980
8000
North Robinson Street 412 336237466 7926
North Robinson Street 524 336237486 386
North Robinson Street 504 336237496 6894
North Robinson Street 526 336237506 8114
Lawnview Avenue 3202 336237543 2277
Edison Highway 1218 336237580 7041
Pelham A

East Monument Street 2813 336250994 7393
East Monument Street 2815 336250995 584
East Monument Street 2811 336250996 6230
North Streeper Street 921 336251015 923
North Streeper Street 907 336251020 7787
North Streeper Street 901 336251023 7142
North Streeper Street 712 336251028 2412
North Streeper Street 702 336251031 3133
North Streeper Street 722 336251035 7634
East Madison Street 2820 336251148 8107
East Madison Street 2816 336251156 5662
North Streeper Street 624 336251224 6174
North Streeper Street 634 336251226 6428
North Streeper Street 632 336251227 6171
North Streeper Street 636 336251230 2089
North Streeper Street 608 336251232 4700
North Streeper Street 616 336251234 3014
North Streeper Street 628 336251236 2095
North Streeper Street 638 336251241 1850
Erdman Avenue 2608 336251291 4358
East Biddle Street 2833 336251498 8534
East Chase Street 2828 336251630 3971
North Streeper Street 904 336251639 2010
North Kenwood Avenue 603 336251740 7536
Ashland Avenue 2802 336251826 833

East Oliver Street 2603 336259984 1229
Llewelyn Avenue 2605 336260006 1622
East Monument Street 2614 336260015 7916
East Monument Street 2616 336260021 2017
East Preston Street 2622 336260042 728
East Biddle Street 2608 336260142 8000
East Monument Street 2613 336260221 143
East Monument Street 2609 336260222 5671
North Luzerne Avenue 929 336260284 3728
Grogan Avenue 2602 336260309 3380
Grogan Avenue 2612 336260313 7455
Grogan Avenue 2604 336260314 706
15000
East Oliver Street 2618 336260554 4021
East Oliver Street 2614 336260563 2138
East Oliver Street 2600 336260564 4053
East Oliver Street 2602 336260566 1242
East Oliver Street 2620 336260572 3955
North Luzerne Avenue 150 336260626 6378
North Luzerne Avenue 140 336260641 1843
North Luzerne Avenue 238 336260759 5749
North Luzerne Avenue 1223 336260805 749
North Luzerne Avenue 1209 336260837 103
North Luzerne Avenue 1207 336260844 100
North Luzerne Avenue 1103 336260876 7822
North Luzerne Avenue 606 336260889 3364
North Luzerne Avenue 

North Milton Avenue 1710 336265996 1915
North Milton Avenue 1708 336266003 2704
North Port Street 818 336266013 6495
North Port Street 804 336266018 1838
North Port Street 824 336266022 3135
East Preston Street 2440 336266033 7000
Ashland Avenue 2409 336266096 6770
Llewelyn Avenue 2413 336266127 8241
North Montford Avenue 427 336266175 158
North Port Street 1707 336266317 5732
North Port Street 1721 336266318 1920
North Montford Avenue 829 336266348 6496
North Montford Avenue 845 336266358 1471
North Montford Avenue 843 336266363 1312
East Hoffman Street 2408 336266393 3915
East Eager Street 2417 336266411 402
East Eager Street 2405 336266412 8387
East Eager Street 2413 336266413 8425
East Eager Street 2403 336266414 1461
East Eager Street 2421 336266415 5772
East Eager Street 2423 336266418 7183
East Lanvale Street 2426 336266468 2711
North Port Street 1819 336266471 2377
North Port Street 1821 336266488 181
North Port Street 1807 336266494 359
North Port Street 1607 336266508 5017
No

Henneman Avenue 2214 336273050 7365
North Collington Avenue 513 336273070 8423
East Oliver Street 2230 336273087 5690
East Chase Street 2207 336273147 5053
East Chase Street 2213 336273150 3607
East Chase Street 2227 336273154 1681
East Chase Street 2233 336273155 2403
East Chase Street 2223 336273157 7396
East North Avenue 2236 336273373 4133
North Duncan Street 5 336273456 5940
East Federal Street 2235 336273462 7319
North Collington Avenue 709 336273502 1292
East Biddle Street 2211 336273768 2510
East Biddle Street 2221 336273772 1109
East Biddle Street 2209 336273774 7923
East Biddle Street 2243 336273779 5622
East Biddle Street 2245 336273780 2579
East Biddle Street 2225 336273785 5011
East Fairmount Avenue 2121 336273997 4706
East North Avenue 2241 336274023 3525
Prentiss Place 2234 336274233 1599
East Oliver Street 2222 336274276 4433
Orleans Street 2132 336274294 5394
Orleans Street 2125 336274359 5074
North Collington Avenue 420 336274539 2421
20000
North Collington Avenue 502

North Washington Street 1924 336288095 1913
North Washington Street 1900 336288098 1416
North Washington Street 1922 336288106 8001
East Lanvale Street 1911 336288143 829
North Chapel Street 1821 336288357 1518
North Gay Street 1424 336288656 7346
North Chapel Street 1835 336288742 1521
North Chapel Street 1825 336288748 5286
North Washington Street 2008 336288906 1902
North Chapel Street 1705 336288960 4393
North Chapel Street 1604 336289311 7424
East Lanvale Street 1910 336289401 6829
East Lanvale Street 1912 336289422 641
Fenwick Avenue 2768 336289769 3137
East Federal Street 1902 336289889 6834
The Alameda 2745 336289895 7184
The Alameda 2729 336289918 797
The Alameda 2711 336289925 5598
The Alameda 2733 336289926 632
23000
East Lanvale Street 1907 336290178 408
North Chapel Street 1830 336290311 1595
North Wolfe Street 1715 336290538 3187
North Wolfe Street 1703 336290542 1037
North Wolfe Street 1811 336290847 7255
North Wolfe Street 1529 336290867 3104
North Wolfe Street 1501 336

North Broadway 1505 336303411 2115
North Broadway 1231 336303498 2110
East Baltimore Street 1632 336303508 4142
Kennedy Avenue 2708 336303647 7020
Kennedy Avenue 2700 336303652 976
Homestead Street 1710 336303838 7129
Homestead Street 1704 336303847 7740
Gorsuch Avenue 1652 336303877 5059
Homestead Street 1707 336303928 1443
Homestead Street 1703 336303929 8068
Homestead Street 1723 336303930 7164
Homestead Street 1715 336303932 175
Cliftview Avenue 1606 336303975 5227
Cliftview Avenue 1602 336303978 8463
Cliftview Avenue 1636 336303985 1991
Cliftview Avenue 1638 336303987 2909
Cliftview Avenue 1616 336304002 7969
Montpelier Street 1757 336304041 2446
North Broadway 1637 336304051 2897
North Broadway 1623 336304053 2663
North Broadway 1629 336304060 8430
North Broadway 1625 336304061 8582
North Broadway 1607 336304069 5164
Gorsuch Avenue 1641 336304113 5075
Gorsuch Avenue 1643 336304122 1033
North Broadway 826 336304134 5305
Kirk Avenue 2919 336304145 6246
Kirk Avenue 2903 336304146 82

28000
North Caroline Street 1308 336315740 4987
Garrett Avenue 2552 336315751 8481
Garrett Avenue 2556 336315756 4664
Garrett Avenue 2520 336315759 4988
Garrett Avenue 2530 336315763 4983
Garrett Avenue 2528 336315765 4986
North Caroline Street 1534 336315916 3733
Aiken Street 2304 336316110 5746
Aiken Street 2308 336316116 7892
Aiken Street 2306 336316119 2666
Aiken Street 2302 336316120 7895
Aiken Street 2318 336316124 3178
East Eager Street 1421 336316217 6806
East Eager Street 1419 336316230 2477
East Eager Street 1441 336316234 1463
North Caroline Street 1836 336316235 1064
Harford Road 2040 336316251 2930
North Spring Street 1211 336316291 8440
North Caroline Street 1616 336316412 5439
North Caroline Street 1634 336316415 3827
Aiken Street 2213 336316465 6919
North Caroline Street 1824 336316524 3795
North Caroline Street 1806 336316547 4888
North Caroline Street 1828 336316567 4881
North Caroline Street 1728 336316665 4843
Cecil Avenue 2600 336316782 6525
Cecil Avenue 2626 33631

In [6]:
print (time.time() - s)/60 , "mins"

163.833625368 mins


In [7]:
len(vacantdb25)

1824

In [8]:
# write data into csv
with open('VacantSet-OSMNX/vacantOSMNX_1516KenhillAve2000_010118', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(vacantOSMNX)
    
"""
# Read data from csv
with open('vacantOSMNX_Baltimore_171117', 'rb') as f:
    reader = csv.reader(f)
    your_list = map(int,list(reader)[0])
""" ;

# Record